In [16]:
import csv
import sqlite3
import pandas as pd

try:
    # Import CSV and extract data
    with open("C:/Users/Adejumod/Desktop/customer_orders.csv", 'r') as fin:
        dr = csv.DictReader(fin)
        customer_db = [(i['order_id'], i['customer_id'], i['order_date'], i['product_category'], i['product_name'], i['quantity'], i['price_per_unit'], i['discount_applied'], i['delivery_status']) for i in dr]

    # Connect to SQLite
    sqliteConnection = sqlite3.connect('etl_staging.db')
    cursor = sqliteConnection.cursor()

    # Create customer Inventory table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS customer (
        order_id VARCHAR(200),
        customer_id VARCHAR(200),
        order_date DATETIME,
        product_category VARCHAR(200),
        product_name VARCHAR(200),
        quantity INTEGER,
        price_per_unit INTEGER,
        discount_applied INTEGER,
        delivery_status VARCHAR(200)
    );
    """)

    # Insert data into the table
    cursor.executemany("""
    INSERT INTO customer (order_id, customer_id, order_date, product_category, product_name, quantity, price_per_unit, discount_applied, delivery_status)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, customer_db)

    # Commit the data to db
    sqliteConnection.commit()

   
    cursor.execute('SELECT * FROM customer limit 5;')
    result = cursor.fetchall()
    

    # Define column names
    columns = ['order_id', 'customer_id', 'order_date', 'product_category', 'product_name', 'quantity', 'price_per_unit', 'discount_applied', 'delivery_status']

    # Convert to DataFrame for tabular display
    df = pd.DataFrame(result, columns=columns)
    print(df)

except sqlite3.Error as error:
    print('Error occurred -', error)

finally:
    if sqliteConnection:
        sqliteConnection.close()
        print('SQLite connection closed')


    order_id customer_id  order_date product_category  product_name  quantity  \
0  ORD-00001    CUST-126  2022-12-31            Books  History Book         1   
1  ORD-00002    CUST-171  2023-07-14       Home Goods       Blender         3   
2  ORD-00003    CUST-140  2023-02-17         Clothing         Dress         1   
3  ORD-00004    CUST-390  2023-08-01      Electronics    Headphones         5   
4  ORD-00005    CUST-311  2023-08-07  Food & Beverage   Protein Bar         1   

   price_per_unit  discount_applied delivery_status  
0          316.12               0.2        Returned  
1           81.15               0.0       Cancelled  
2          227.48               0.0         Pending  
3          169.12               0.0       Delivered  
4          380.31               0.0         Shipped  
SQLite connection closed
